## 说明

输入： ① 四个训练好的LSTM网络，对应四个传感器

……… ② 四个数据集，内容为四个传感器信号的特征矩阵

     
函数： ① my_transform，对数据集进行预处理

……… ② weights，计算决策矩阵权重

     
输出： ① 决策融合后的识别精度

作者：许志翔（西安交通大学 车辆71）

联系方式：mr_xuzhixiang@qq.com

参考：孙玉林，余本国.PyTorch深度学习入门与实战.北京:水利水电出版社,2020.

………杨纶标, 高英仪, 凌卫新. 模糊数学原理及应用.第5版[M]. 华南理工大学出版社, 2011.

In [3]:
import torch
import torch.nn as nn
import numpy as np
import seaborn as sns # 绘图
import torchvision.transforms as transforms
import torch.utils.data as Data
from scipy.io import loadmat # mat数据加载接口
from sklearn.model_selection import train_test_split # 用于数据集的切分
from sklearn.preprocessing import StandardScaler,MinMaxScaler # 数据标准化预处理
from sklearn.metrics import accuracy_score,confusion_matrix # 用于计算精度与混淆矩阵
from sklearn.manifold import TSNE # 降维
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# 1. 加载数据集（传感器1到4）
file1 = 'D:\My files\diploma project\data\data_1.mat'
file2 = 'D:\My files\diploma project\data\data_2.mat'
file3 = 'D:\My files\diploma project\data\data_3.mat'
file4 = 'D:\My files\diploma project\data\data_4.mat'

In [5]:
## mat_dtype=True，保证了导入后变量的数据类型与原类型一致。
data_1 = loadmat(file1, mat_dtype=True)
data_2 = loadmat(file2, mat_dtype=True)
data_3 = loadmat(file3, mat_dtype=True)
data_4 = loadmat(file4, mat_dtype=True)

In [6]:
## 导入后的data是一个字典，取出想要的变量字段即可。
X1 = data_1['Features_1']
Y1 = data_1['targets_1']

X2 = data_2['Features_2']
Y2 = data_2['targets_2']

X3 = data_3['Features_3']
Y3 = data_3['targets_3']

X4 = data_4['Features_4']
Y4 = data_4['targets_4']

In [7]:
## 2. 自定义数据集处理函数
def my_transform(num_x, num_y):
    X_p = np.transpose(num_x,(2,0,1))
    X_p = X_p.reshape(700,1640)
    ## Y_p = Y.reshape(Y.shape[1])不知道为什么会报错
    ## Y_p这里reshape成700×1而非700是为了解决torch.cat报错
    ## RuntimeError: zero-dimensional tensor (at position 1) cannot be concatenated
    Y_p = num_y.reshape(700,1)
    
    ## 将X,Y转为张量
    x = torch.from_numpy(X_p.astype(np.float32))
    y = torch.from_numpy(Y_p.astype(np.int64))
    
    ## 将数据随机切分为训练集75%和测试集25%
    ## x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=123)
    ## 在全数据集上测试，故不切分数据集
    
    ## 数据归一化，特征值范围转化到0~1间
    ## 注意，只能对训练集使用fit
    scales = MinMaxScaler(feature_range=(0,1))
    x_all_s = scales.fit_transform(x)

    ## 归一化后X又变成numpy了，再重新转成tensor
    x_all_s = torch.from_numpy(x_all_s.astype(np.float32))
    
    ## return sets
    return x_all_s, y

In [8]:
## 3. 定义权重计算函数，最大偏差模型（王应明）

## 输入决策矩阵，7×4 tensor矩阵
def weights(dm):
    ## 决策矩阵去量纲
    scales = MinMaxScaler(feature_range=(0,1))
    dm_s = scales.fit_transform(dm.detach())
    
    summat = np.zeros(dm_s.shape) ## 代表对应位置处元素与所在列的差的和
    W = np.zeros(dm_s.shape[1]) ## 权重向量
    for j in range(dm_s.shape[1]):
        for i in range(dm_s.shape[0]):
            for k in range(dm_s.shape[0]):
                summat[i,j] += abs(dm_s[i,j]-dm_s[k,j])
            
    for j in range(dm_s.shape[1]):
        V = sum(summat,0)
        W[j] = V[j]/sum(V)
    
    ## 输出权重向量
    return W

In [9]:
## 4. 数据预处理
## sensor1代表传感器1获得的信号特征矩阵，y1表示对应的标签
sensor1,y1 = my_transform(X1, Y1)
sensor2,y2 = my_transform(X2, Y2)
sensor3,y3 = my_transform(X3, Y3)
sensor4,y4 = my_transform(X4, Y4)

## 比较四个传感器标签是否一致，一致则统一用y表示
if (y1==y2).all()&(y2==y3).all()&(y3==y4).all():
    y = y1
else:
    print("标签错误")

print(sensor1.shape)
print(y.shape)

torch.Size([700, 1640])
torch.Size([700, 1])


In [10]:
## 5. 搭建LSTM分类器
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim ## LSTM神经元个数
        self.layer_dim = layer_dim   ## LSTM的层数
        self.lstm = nn.LSTM(input_dim,hidden_dim,layer_dim,batch_first=True) ## nonlinearity='relu'
        ## 连接全连接层
        self.fc1 = nn.Linear(hidden_dim,output_dim) ## self.fc1 = nn.Linear(hidden_dim,output_dim)
        ## 添加dropout层
        ## self.dropout = nn.Dropout(p=0.5)  ## dropout训练
        
    def forward(self,x):
        ## x:[batch,time_step,input_dim]
        ## h_n:[layer_dim,batch,hidden_dim]
        h_0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().to(device)
        c_0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().to(device)
        out, (h_n, c_n) = self.lstm(x,(h_0.detach(),c_0.detach())) ## None表示h0会使用全0进行初始化
        ## 选取最后一个时间点的out输出
        ## out = self.dropout(out)
        out = self.fc1(out[:,-1,:])
        return out

In [11]:
## 6. 初始化模型
input_dim=40 ## 图片每行的像素数量
hidden_dim=64 ## RNN神经元个数
layer_dim=1 ## RNN的层数
output_dim=7 ## 隐藏层输出的维度（7类图像）
sequence_dim = 41 ## 序列长度（矩阵有多少行）

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 初始化传感器1—4的lstm模型
lstm1 = LSTM(input_dim,hidden_dim,layer_dim,output_dim)
lstm2 = LSTM(input_dim,hidden_dim,layer_dim,output_dim)
lstm3 = LSTM(input_dim,hidden_dim,layer_dim,output_dim)
lstm4 = LSTM(input_dim,hidden_dim,layer_dim,output_dim)

# 模型参数地址
path1 = "D:\My files\diploma project\python codes\lstm1.pkl"
path2 = "D:\My files\diploma project\python codes\lstm2.pkl"
path3 = "D:\My files\diploma project\python codes\lstm3.pkl"
path4 = "D:\My files\diploma project\python codes\lstm4.pkl"

# 加载模型参数
lstm1.load_state_dict(torch.load(path1)) 
lstm2.load_state_dict(torch.load(path2)) 
lstm3.load_state_dict(torch.load(path3)) 
lstm4.load_state_dict(torch.load(path4)) 

<All keys matched successfully>

In [14]:
## 7. 模型验证
lstm2.eval()
test_y_all = torch.LongTensor()
pre_lab_all = torch.LongTensor()

## 使用sensor4验证一下看看模型正不正常
for b_x, b_y in zip (sensor2,y):
    ## input :[batch, time_step, input_dim]
    xdata = b_x.view(-1, sequence_dim, input_dim)
    output = lstm2(xdata)
    pre_lab = torch.argmax(output,1)
    test_y_all = torch.cat((test_y_all,b_y)) ## 测试集的标签
    pre_lab_all = torch.cat((pre_lab_all,pre_lab)) ## 测试集的预测标签
acc = accuracy_score(test_y_all,pre_lab_all)
print("传感器2的预测精度为：",acc)

print(test_y_all.shape)
print(pre_lab_all.shape)

传感器2的预测精度为： 0.9157142857142857
torch.Size([700])
torch.Size([700])


In [11]:
## 8.生成决策矩阵
for i in range(y.shape[0]):
    xdata1 = sensor1[i].view(-1, sequence_dim, input_dim)
    xdata2 = sensor2[i].view(-1, sequence_dim, input_dim)
    xdata3 = sensor3[i].view(-1, sequence_dim, input_dim)
    xdata4 = sensor4[i].view(-1, sequence_dim, input_dim)
    
    output1 = lstm1(xdata1)
    output2 = lstm2(xdata2)
    output3 = lstm3(xdata3)
    output4 = lstm4(xdata4)
    
    ## 决策矩阵 decision matrix 行代表各标签隶属度，列代表传感器
    DM = torch.cat((output1,output2,output3,output4)) 
    DM = torch.transpose(DM,1,0)
    ## 决策矩阵去量纲
    scales = MinMaxScaler(feature_range=(0,1))
    DM = scales.fit_transform(DM.detach())
    DM = torch.from_numpy(DM)
    
    if i == 0:
        dmset = DM
    else:
        dmset = torch.cat((dmset,DM),dim=0)
        
## 决策矩阵集        
DMSET = dmset.reshape(700,7,4)


In [12]:
## 9.决策融合
PRE_LAB_ALL = torch.LongTensor()

for i in range(DMSET.shape[0]):
    W = weights(DMSET[i]) ## 权重向量
    D_mat = DMSET[i] * W ## 评价矩阵 = 决策矩阵×权重
    D = torch.sum(D_mat,1) ## 融合后的评估值
    PRE_LAB = torch.argmax(D)
    PRE_LAB = PRE_LAB.reshape(1)
    PRE_LAB_ALL = torch.cat((PRE_LAB_ALL,PRE_LAB)) ## 预测的标签

y = y.reshape(700)
ACC = accuracy_score(y,PRE_LAB_ALL)  
print("决策融合后的识别精度为：",ACC*100,"%")

决策融合后的识别精度为： 100.0 %
